In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.append(os.path.abspath("."))
import wandb
import pandas as pd
from dmsensei.config import device
from dmsensei import DataModule, create_model
import sys
import os


dm = DataModule(
    name=["yack_train"], # finetune: "utr", "pri_miRNA", "archiveII"
    strategy='random', #random, sorted or ddp
    shuffle_train=False,
    data_type=["dms", "shape", "structure"],  #
    force_download=False,
    batch_size=1,
    max_len=1024,
    structure_padding_value=0,
    train_split=None,
    external_valid=["yack_valid", "utr", "pri_miRNA", "human_mRNA"], # finetune: "yack_valid", "human_mRNA"
)
dm.setup('fit')

model = create_model(
    model="cnn",
    ntoken=5,
    d_model=64,
    d_cnn=128,
    n_heads=16,
    dropout=0,
    lr=1e-4,
    weight_decay=0,
    gamma=0.995,
).to(device)

/root/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            
Loading dataset from disk
Done!                            


In [2]:
import tqdm
import torch 
import time

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0)
timings = []
for epoch in range(100):    
    for idx, batch in tqdm.tqdm(enumerate(dm.train_dataloader()), desc="Epoch {}".format(epoch), total=len(dm.train_dataloader())):
        tic = time.time()
        batch.to(device)
        pred = model.forward(batch)
        batch.integrate_prediction(pred)
        loss = model.loss_fn(batch)[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        timings.append(time.time() - tic)

Epoch 0:   1%|          | 714/111960 [02:33<6:38:26,  4.65it/s] 


KeyboardInterrupt: 